<a href="https://colab.research.google.com/github/aheiX/Teaching/blob/main/Job%20Shop%20Scheduling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Orienteering Problem

## Mathematical Model

$
\begin{align}
  \begin{array}{lll}
    &\textbf{Objective} & \\
    & \max \sum\limits_{i=2}^{N-1} \sum\limits_{j=2}^{N} S_{i} \cdot x_{ij} &~~~ (0) \\
    &&\\
    &\textbf{Constraints} & \\
    & \sum\limits_{j=2}^{N} x_{1j} = 1  &~~~ (1a) \\
    & \sum\limits_{i=1}^{N-1} x_{iN} = 1  &~~~ (1b) \\
    & \sum\limits_{i=1}^{N-1} x_{ik} \le 1,~ \forall~ k = 2,\dots,N-1  &~~~ (2a) \\
    & \sum\limits_{i=1}^{N-1} x_{ik} = \sum\limits_{j=2}^{N} x_{kj},~ \forall~ k = 2,\dots,N-1  &~~~ (2b) \\
    & \sum\limits_{i=1}^{N-1} \sum\limits_{j=2}^{N} t_{ij} \cdot x_{ij} \le T_{max} &~~~ (3) \\
    & u_{i} \in \{2, 3, ... N \},~ \forall~ i = 2,\dots,N   &~~~ (4) \\
    & u_{i} - u_{j} + 1 \le (N -1) \cdot (1-x_{ij}),~ \forall~ i,j = 2,\dots,N  &~~~ (5) \\
    & x_{ij} \in \{0,1\},~ \forall~ i,j = 1,\dots,N  &~~~ (6) \\
  \end{array}
\end{align}
$
<br><br>
Based on: Vansteenwegen, P., Souffriau, W., & Van Oudheusden, D. (2011). The orienteering problem: A survey. *European Journal of Operational Research*, 209(1), 1-10.

## Python implementation

### Artificial data

In [ ]:
import numpy as np

# USER INPUT
seed = 1
number_of_nodes = 6
T_max = int(25 * (number_of_nodes - 2)) # On avarage, half of the nodes can be visited

#################

# set seed
np.random.seed(seed)

# nodes: a python list
nodes = ['node ' + str(i) for i in range(number_of_nodes)]

# score: a python dict with each node having an entry
S = {node: np.random.randint(1,100) for node in nodes}

# times: a python dict with each node-node combination having an entry
t = {n1: {n2: np.random.randint(1,100) for n2 in nodes}
     for n1 in nodes}

# print data
print(S)
print(t)
print(T_max)

{'node 0': 38, 'node 1': 13, 'node 2': 73, 'node 3': 10, 'node 4': 76, 'node 5': 6}
{'node 0': {'node 0': 80, 'node 1': 65, 'node 2': 17, 'node 3': 2, 'node 4': 77, 'node 5': 72}, 'node 1': {'node 0': 7, 'node 1': 26, 'node 2': 51, 'node 3': 21, 'node 4': 19, 'node 5': 85}, 'node 2': {'node 0': 12, 'node 1': 29, 'node 2': 30, 'node 3': 15, 'node 4': 51, 'node 5': 69}, 'node 3': {'node 0': 88, 'node 1': 88, 'node 2': 95, 'node 3': 97, 'node 4': 87, 'node 5': 14}, 'node 4': {'node 0': 10, 'node 1': 8, 'node 2': 64, 'node 3': 62, 'node 4': 23, 'node 5': 58}, 'node 5': {'node 0': 2, 'node 1': 1, 'node 2': 61, 'node 3': 82, 'node 4': 9, 'node 5': 89}}
100


In [ ]:
# demonstration of how to access nodes
print(nodes)          # all nodes
print(nodes[1:])      # all but first node
print(nodes[:-1])     # all but last node
print(nodes[1:-1])    # all but first and last node

['node 0', 'node 1', 'node 2', 'node 3', 'node 4', 'node 5']
['node 1', 'node 2', 'node 3', 'node 4', 'node 5']
['node 0', 'node 1', 'node 2', 'node 3', 'node 4']
['node 1', 'node 2', 'node 3', 'node 4']


### Packages

In [ ]:
!pip install pulp
import pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Model

In [ ]:
# model
model = pulp.LpProblem(name='Orienteering_Problem', sense=pulp.constants.LpMaximize)

# decision variables
x = pulp.LpVariable.dicts(name='x', indices=(nodes, nodes), cat='Binary')
u = pulp.LpVariable.dicts(name='u', indices=nodes[1:],
                          lowBound=2, upBound=len(nodes),
                          cat='Integer')

# objective
model += pulp.lpSum(S[i]*x[i][j] for i in nodes[1:-1] for j in nodes[1:]), '(0)'

# (1a)
model += pulp.lpSum(x[nodes[0]][j] for j in nodes[1:]) == 1, '(1a)'

# (1b)
model += pulp.lpSum(x[i][nodes[-1]] for i in nodes[:-1]) == 1, '(1b)'

for k in nodes[1:-1]:
  # (2a)
  model += pulp.lpSum(x[i][k] for i in nodes[:-1]) <= 1, '(2a):' + k
  # (2b)
  model += pulp.lpSum(x[i][k] for i in nodes[:-1]) <= pulp.lpSum(x[k][j] for j in nodes[1:]), '(2b):' + k

# (3)
model += pulp.lpSum(t[i][j] * x[i][j] for i in nodes[:-1] for j in nodes[1:]) <= T_max, '(3)'

# (4) -> see initialization of decision variable u

# (5)
for i in nodes[1:]:
  for j in nodes[1:]:
    model += u[i] - u[j] + 1 <= (len(nodes) - 1) * (1 - x[i][j]), '(5):' + i + j

# (6) -> see initialization of decision variable x



### Solution

In [ ]:
# solve problem
model.solve()

# get status
print("Status:", pulp.LpStatus[model.status])

# get objective value
print('Objective value:', round(pulp.value(model.objective), 2))

# get tour
print('Tour: ')
t_sum = 0
i = nodes[0]
while i != nodes[-1]:
  for j in nodes:
    try:
      if x[i][j].varValue > 0:
        print(' ' + str(x[i][j]) + ': ' + str(x[i][j].varValue) + ', t_ij=' + str(t[i][j]) + ', S_j=' + str(S[j]))
        t_sum += t[i][j]
        i = j
        break
    except:
      pass
print(' total time: ' + str(t_sum))

# get value of decision variable u (position in tour of the nodes that are part of the tour)
print('')
print('u-variable: ')
for i in nodes[1:]:
  print(' ' + str(u[i]) + ': ' + str(u[i].varValue))

Status: Optimal
Objective value: 96.0
Tour: 
 x_node_0_node_2: 1.0, t_ij=17, S_j=73
 x_node_2_node_1: 1.0, t_ij=29, S_j=13
 x_node_1_node_3: 1.0, t_ij=21, S_j=10
 x_node_3_node_5: 1.0, t_ij=14, S_j=6
 total time: 81

u-variable: 
 u_node_1: 3.0
 u_node_2: 2.0
 u_node_3: 4.0
 u_node_4: 2.0
 u_node_5: 6.0
